# 1. 패키지 설치

In [1]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Knowledge Base 구성을 위한 데이터 생성

- Chroma를 활용한 [3.2 LangChain과 Chroma를 활용한 RAG 구성](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.2%20LangChain%EA%B3%BC%20Chroma%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20RAG%20%EA%B5%AC%EC%84%B1.ipynb)과 동일함
- Vector Database만 [Pinecone](https://www.pinecone.io/)으로 변경

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_docs/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
document_list[52]

Document(metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ①

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [5]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [6]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

In [8]:
pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [9]:
query = '연봉 5천만원인 거주자의 종합소득세는?'

# 3. 답변 생성을 위한 Retrieval

- `RetrievalQA`에 전달하기 위해 `retriever` 생성
- `search_kwargs` 의 `k` 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- `.invoke()` 를 호출해서 어떤 문서를 가져오는지 확인 가능

In [10]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='ea2c5c42-74eb-4762-86d8-21c1380dd2c0', metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\Python310\lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [13]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [14]:
# LangChain 권장사항에 따라 강의 코드와 다르게 `.invoke()`를 사용합니다
ai_message = qa_chain.invoke({"query": query})

In [15]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉 5천만원인 거주자의 종합소득세는 624만원입니다. 이는 5천만원 초과 금액에 대한 세율이 적용되지 않기 때문입니다.'}